In [8]:
import numpy as np
import random
import copy
SIZE=5
GAMMA=0.5
ALL_ACTIONS= ('U', 'D', 'L', 'R','X')
positions=()
board=()
jean=(4,1) #Initial position of jean 
magneto=() 
wolve=() 
m_not_allow=(4,4) #magneto can't move to pos 5,5, ie (4,4) intex
blocked=(3,2) #blocked position due to wall
def give_actions(i,j,player): #it will return the valid actions of the particular player(using for wolve only)
    global SIZE
    flag=[1]*4 #U
    if (i-1==blocked[0] and j==blocked[1]) or i-1<0 or (i-1,j)==player:
        flag[0]=0 #U
    if (i+1==blocked[0] and j==blocked[1]) or i+1>=SIZE or (i+1,j)==player:
        flag[1]=0 #D
    if (j-1==blocked[1] and i==blocked[0]) or j-1<0 or (i,j-1)==player:
        flag[2]=0 #L
    if (j+1==blocked[1] and i==blocked[0]) or j+1>=SIZE or (i,j)==player:
        flag[3]=0 #R
    temp=[]
    if flag[0]==1:
        temp.append('U')
    if flag[1]==1:
        temp.append('D')
    if flag[2]==1:
        temp.append('L')
    if flag[3]==1:
        temp.append('R')
#     if any(flag):
    temp.append('X') #appending the no move condition
    return tuple(temp)

def make_move(z,action): #will update cordinates w.r.t to action
    if action == 'U':
        z[0] -= 1
    elif action == 'D':
        z[0] += 1
    elif action == 'R':
        z[1] += 1
    elif action == 'L':
        z[1] -= 1
    return tuple(z)

def move_jean_magnato():
    global magneto,jean,wolve
    if random.random()<=0.20:  #jean remain at its position 80% of time
        if jean==(0,4):
            jean=(4,1)
        else:
            jean=(0,4)
            
    z=copy.deepcopy(magneto)
    if random.random()<=0.95: #probability to move to other position
        for i in range(50): #we need a valid value (not (4,4)) for the magneto
            valid_moves=give_actions(magneto[0],magneto[1],wolve)
            valid_moves=np.random.choice(valid_moves)
            z=make_move(list(z),valid_moves)
            if z==m_not_allow:  #magneto can't move to index (4,4) ie location 5,5
                continue
            break
    magneto=z
    
class Grid: #SIZE x SIZE grid object
  def __init__(self,size,loc):
    self.size=size
    self.x =loc[0]
    self.y=loc[1]   #updating the starting position x,y coordinates

  def set(self,rewards,actions):
    self.rewards=rewards  #setting the rewards of the object
    self.actions=actions #setting the actions of the object

  def current_state(self): #return the current state of the object (wolve)
    return (self.x, self.y)

  def set_state(self,state):
    self.x=state[0]
    self.y=state[1]
    global wolve
    wolve=(self.x,self.y) #updating the latest wolve. position


  def move(self, act):  #will update coordinates
    if random.random()<=0.05: #%% of the time, it don't move at all
        return self.rewards.get((self.x,self.y),0)
    if act in self.actions[(self.x, self.y)]:
      if act=='U': #UP
        self.x-=1
      elif act=='D': #DOWN
        self.x+=1
      elif act=='R': #RIGHT
        self.y+=1
      elif act=='L': #LEFT
        self.y-=1
    #if 'X' found, no update to the x and y coordinates
    return self.rewards.get((self.x,self.y),0)

  def all_states(self):
    return  set(self.rewards.keys())| set(self.actions.keys())

def standard_grid():
  global ALL_ACTIONS,blocked,wolve,SIZE,magneto,jean
  n=SIZE
  for i in range(200): #will try to generate the valid non intersecting positions FOR MAGNETO wolve during 
                        #initialization of the board
        magneto=(random.randint(0,n-1),random.randint(0,n-1))  #random position for magneto
        wolve=(random.randint(0,n-1),random.randint(0,n-1))   #random position for wolve
        if magneto==wolve or magneto==blocked or wolve==blocked or wolve==jean or magneto==jean or magneto==m_not_allow:
            continue   #if not valid position for wolve and magneto, continue
        break
  g_obj= Grid(SIZE,wolve)  #creating the object
  rewards = {}
  actions={}
  for i in range(SIZE):
        for j in range(SIZE):
            if (i,j)==blocked: # or (i,j)==jean or (i,j)==magneto:
                continue
            act='X'
            if random.random()<=0.95: #95% of time it will move to different position
                act=give_actions(i,j,magneto)
            actions[(i,j)]=act #updating the actions
  g_obj.set(rewards, actions)  #updating actions (all actions added)
  return g_obj #returning object


def create_grid(default_cost=0.0):  #default cost (reward) of all position is 0
  g_obj = standard_grid()
  t_rwd={} #temp rewards
  for i in range(SIZE):
        for j in range(SIZE):
            t_rwd[(i,j)]=default_cost
            
  ##updating the rewards w.r.t jean, magneto and wolve
  t_rwd[jean]=20
  t_rwd[magneto]=-20
  if jean==magneto and magneto==wolve:
    t_rwd[magneto]=-15
  g_obj.rewards.update(t_rwd)
  return g_obj

def print_values(Val, grid):
  for i in range(grid.size):
    print("-----------------------------------")
    for j in range(grid.size):
      value = Val.get((i,j), 0)
      if value >= 0:
        print(" %.2f|" % value, end="")
      else:
        print("%.2f|" % value, end="")
    print()


def print_policy(Pol, grid):
  for i in range(grid.size):
    print("-------------------------------")
    for j in range(grid.size):
      action = Pol.get((i,j), ' ')
      print("  %s  |" % action, end="")
    print()

In [9]:
grid = create_grid()
print("Initial rewards configurations:")
print_values(grid.rewards, grid)

Initial rewards configurations:
-----------------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|
-----------------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|
-----------------------------------
 0.00| 0.00| 0.00| 0.00| 0.00|
-----------------------------------
 0.00| 0.00| 0.00| 0.00|-20.00|
-----------------------------------
 0.00| 20.00| 0.00| 0.00| 0.00|


In [10]:
policy ={} #policy
for st in grid.actions.keys():
  policy[st]=np.random.choice(ALL_ACTIONS) #updating the policies (choosing randomly) for wolve

In [11]:
print("Initial random policy")
print_policy(policy, grid)

Initial random policy
-------------------------------
  R  |  R  |  U  |  D  |  L  |
-------------------------------
  D  |  R  |  D  |  D  |  R  |
-------------------------------
  L  |  D  |  D  |  L  |  D  |
-------------------------------
  U  |  D  |     |  U  |  D  |
-------------------------------
  D  |  X  |  X  |  D  |  R  |


In [12]:
#print(grid.all_states())

In [13]:
Val = {} #value 
states=grid.all_states()
for st in states:
  if st in grid.actions:
    Val[st] = np.random.random()  #randomly initializing the values
  else:
    Val[st] = 0   #blocked wall will be having zero probability
print("Initial random probability")
print_values(Val, grid)

Initial random probability
-----------------------------------
 0.57| 1.00| 0.64| 0.87| 0.52|
-----------------------------------
 0.94| 0.15| 0.71| 0.35| 0.65|
-----------------------------------
 0.21| 0.29| 0.93| 0.17| 0.63|
-----------------------------------
 0.74| 0.27| 0.00| 0.41| 0.04|
-----------------------------------
 0.62| 0.68| 0.85| 0.08| 0.08|


In [14]:
iter=0
for _ in range(200):
  iter+=1
  print()
  print("Values %d: " % iter)
  print_values(Val, grid)
  print()
  print("Policy %d: " % iter)
  print_policy(policy, grid)

  delta_change=0
  ##POLICY EVALUATIONS
  for __ in range(100):
      for st in states:
        
        old_val = Val[st]
        if st!=blocked:   #we are not calculating policy for wall
            old_pol= policy[st]
        new_pol=None
        if st in policy: 
          a=policy[st]
          grid.set_state(st)
          r =grid.move(a)  
          c=grid.current_state()
          if c==jean or c==magneto:    #if current state is jean or magneto, then v will not change
            v=r
          else:
            v = r + GAMMA*Val[c]
            
          Val[st] = v       ##VALUE UPDATE 

          #######MOVING THE MAGNATO AND JEAN AND UPDATING THE REWARDS FOR WOLVE (for same state) ###
          grid.rewards[jean]=0
          grid.rewards[magneto]=0

          move_jean_magnato()      #it will move both jean and magnato to their respective new positions
        
          grid.rewards[jean]=20
          grid.rewards[magneto]=-20

          if jean==magneto and wolve==jean:
             grid.rewards[jean]=-15

          delta_change = max(delta_change, np.abs(old_val - Val[st]))
          if delta_change < 0.001:
            break
  
  ###POLICY IMPROVEMENT STEP###
  policy_converged=True
  for st in policy:
      if st in states:
        old_act=policy[st]
        new_act= None
        new_val = float('-inf')
        for a in give_actions(st[0],st[1],magneto):   #checking particular state agains it's its best actions
            grid.set_state(st)
            r = grid.move(a) 
            c=grid.current_state()
            if c==jean or c==magneto:    #if current state is jean or magneto, then v will not change
                v=r
            else:
                v = r + GAMMA * Val[c]
            if v > new_val:
              new_val = v
              new_act = a
                
            policy[st]=new_act
            
        if new_act!=old_act:
           policy_converged=False
        
        grid.rewards[jean]=0
        grid.rewards[magneto]=0

        move_jean_magnato()      #it will move both jean and magnato to their respective new positions

        grid.rewards[jean]=20
        grid.rewards[magneto]=-20
          
        if jean==magneto and wolve==jean:
            grid.rewards[jean]=-15
  if policy_converged:
    break


Values 1: 
-----------------------------------
 0.57| 1.00| 0.64| 0.87| 0.52|
-----------------------------------
 0.94| 0.15| 0.71| 0.35| 0.65|
-----------------------------------
 0.21| 0.29| 0.93| 0.17| 0.63|
-----------------------------------
 0.74| 0.27| 0.00| 0.41| 0.04|
-----------------------------------
 0.62| 0.68| 0.85| 0.08| 0.08|

Policy 1: 
-------------------------------
  R  |  R  |  U  |  D  |  L  |
-------------------------------
  D  |  R  |  D  |  D  |  R  |
-------------------------------
  L  |  D  |  D  |  L  |  D  |
-------------------------------
  U  |  D  |     |  U  |  D  |
-------------------------------
  D  |  X  |  X  |  D  |  R  |

Values 2: 
-----------------------------------
-0.00|-0.00|-0.00|-5.00|-0.62|
-----------------------------------
-0.08|-0.62|-1.25|-10.00|-0.04|
-----------------------------------
-0.08| 10.00|-2.50|-1.25|-0.00|
-----------------------------------
-20.00| 20.00| 0.00|-10.00| 0.00|
-----------------------------------
-1.25


Values 14: 
-----------------------------------
-1.25| 1.25| 2.50| 2.50| 5.00|
-----------------------------------
-2.50| 2.50| 1.25| 2.50| 2.50|
-----------------------------------
 5.00| 5.00| 5.00|-20.00| 10.00|
-----------------------------------
 10.00| 10.00| 0.00| 5.00| 0.62|
-----------------------------------
 20.00| 20.00| 10.00| 10.00| 5.00|

Policy 14: 
-------------------------------
  R  |  D  |  R  |  R  |  X  |
-------------------------------
  D  |  D  |  D  |  U  |  U  |
-------------------------------
  D  |  D  |  L  |  R  |  D  |
-------------------------------
  D  |  D  |     |  D  |  U  |
-------------------------------
  R  |  X  |  L  |  L  |  L  |

Values 15: 
-----------------------------------
-10.00|-20.00| 5.00| 5.00| 10.00|
-----------------------------------
 0.31| 0.31| 0.31| 5.00| 5.00|
-----------------------------------
 5.00| 0.62| 0.62|-0.00|-0.00|
-----------------------------------
 10.00| 1.25| 0.00| 0.62|-0.00|
-------------------------------

Values 27: 
-----------------------------------
 2.50| 5.00| 10.00| 5.00| 10.00|
-----------------------------------
-0.01|-0.01| 5.00| 10.00| 5.00|
-----------------------------------
-0.01|-0.02|-0.02| 5.00| 10.00|
-----------------------------------
-0.01|-0.04| 0.00| 2.50| 5.00|
-----------------------------------
 10.00| 20.00| 10.00| 1.25| 0.62|

Policy 27: 
-------------------------------
  R  |  R  |  D  |  D  |  X  |
-------------------------------
  U  |  U  |  U  |  U  |  U  |
-------------------------------
  U  |  U  |  U  |  U  |  D  |
-------------------------------
  D  |  D  |     |  U  |  U  |
-------------------------------
  R  |  X  |  L  |  L  |  L  |

Values 28: 
-----------------------------------
-0.31|-0.31|-0.62|-0.00| 10.00|
-----------------------------------
-0.31|-0.31|-0.31|-0.00| 20.00|
-----------------------------------
-0.16|-0.31|-0.62|-0.00|-0.00|
-----------------------------------
 10.00| 20.00| 0.00|-0.00|-0.00|
---------------------------------

Values 39: 
-----------------------------------
-0.00|-0.00| 5.00| 10.00| 20.00|
-----------------------------------
-0.01|-0.00|-0.00|-0.00|-0.00|
-----------------------------------
-0.00|-0.00|-0.00|-0.00|-0.00|
-----------------------------------
-0.00|-0.00| 0.00|-0.00|-0.00|
-----------------------------------
 20.00| 1.25| 20.00| 10.00|-0.00|

Policy 39: 
-------------------------------
  R  |  D  |  X  |  L  |  X  |
-------------------------------
  R  |  D  |  U  |  D  |  D  |
-------------------------------
  D  |  R  |  R  |  D  |  L  |
-------------------------------
  D  |  D  |     |  D  |  D  |
-------------------------------
  R  |  X  |  L  |  U  |  L  |

Values 40: 
-----------------------------------
-0.08|-0.16|-20.00|-10.00| 10.00|
-----------------------------------
-0.08|-0.04|-10.00|-0.16|-0.16|
-----------------------------------
 1.25|-0.08|-0.16|-0.16|-0.31|
-----------------------------------
 2.50| 5.00| 0.00|-0.31|-0.31|
-----------------------------------

Values 54: 
-----------------------------------
-1.25|-2.50|-5.00|-5.00|-5.00|
-----------------------------------
-1.25|-1.25|-2.50|-5.00| 20.00|
-----------------------------------
-0.62|-1.25|-2.50|-2.50|-5.00|
-----------------------------------
-0.04| 10.00| 0.00|-0.16|-5.00|
-----------------------------------
-0.04| 10.00| 20.00|-20.00|-0.00|

Policy 54: 
-------------------------------
  D  |  D  |  D  |  R  |  X  |
-------------------------------
  D  |  D  |  L  |  R  |  U  |
-------------------------------
  D  |  D  |  L  |  D  |  U  |
-------------------------------
  R  |  D  |     |  X  |  D  |
-------------------------------
  R  |  R  |  X  |  L  |  U  |

Values 55: 
-----------------------------------
 1.25| 2.50| 1.25| 2.50| 5.00|
-----------------------------------
 2.50|-0.00| 2.50| 10.00| 2.50|
-----------------------------------
-0.00|-0.00| 5.00|-0.00| 10.00|
-----------------------------------
-0.00|-0.00| 0.00|-0.00|-0.00|
-----------------------------------
-

...

Values 143: 
-----------------------------------
-0.00|-0.00|-0.00| 2.50| 2.50|
-----------------------------------
-0.00|-0.00|-0.00|-0.31|-0.31|
-----------------------------------
-0.00|-0.00|-0.00|-0.31|-0.62|
-----------------------------------
-10.00|-0.00| 0.00|-0.31|-0.62|
-----------------------------------
-20.00|-20.00| 1.25| 2.50|-0.00|

Policy 143: 
-------------------------------
  X  |  D  |  R  |  R  |  L  |
-------------------------------
  U  |  X  |  L  |  U  |  U  |
-------------------------------
  X  |  U  |  U  |  U  |  U  |
-------------------------------
  U  |  U  |     |  D  |  D  |
-------------------------------
  R  |  X  |  L  |  X  |  L  |

Values 144: 
-----------------------------------
-0.00|-0.00| 2.50| 20.00| 2.50|
-----------------------------------
-0.00|-0.00|-0.00| 2.50| 20.00|
-----------------------------------
-0.00|-0.00|-0.00| 1.25| 2.50|
-----------------------------------
-0.00|-0.00| 0.00|-0.62|-0.62|
-----------------------------------


  R  |  X  |  R  |  U  |  U  |
-------------------------------
  R  |  D  |  L  |  U  |  U  |
-------------------------------
  R  |  D  |     |  D  |  U  |
-------------------------------
  X  |  X  |  L  |  L  |  L  |

Values 156: 
-----------------------------------
-0.00|-0.00|-10.00| 10.00| 20.00|
-----------------------------------
-0.00|-0.00|-5.00|-10.00| 10.00|
-----------------------------------
 0.62| 1.25| 1.25|-5.00| 10.00|
-----------------------------------
 1.25| 2.50| 0.00| 1.25|-10.00|
-----------------------------------
-0.01| 2.50| 2.50| 2.50| 1.25|

Policy 156: 
-------------------------------
  R  |  L  |  R  |  R  |  X  |
-------------------------------
  D  |  D  |  D  |  U  |  U  |
-------------------------------
  D  |  D  |  L  |  D  |  D  |
-------------------------------
  R  |  D  |     |  D  |  U  |
-------------------------------
  R  |  U  |  L  |  L  |  L  |

Values 157: 
-----------------------------------
-0.00|-0.00|-10.00| 5.00| 10.00|
------------


Values 168: 
-----------------------------------
-0.08|-0.16|-0.31|-0.02|-0.04|
-----------------------------------
-0.08|-0.08|-0.16|-0.04|-0.04|
-----------------------------------
-20.00|-10.00|-0.16|-0.02|-0.04|
-----------------------------------
 2.50| 20.00| 0.00|-0.02|-0.04|
-----------------------------------
 5.00|-20.00| 5.00|-0.02|-0.00|

Policy 168: 
-------------------------------
  D  |  D  |  R  |  X  |  L  |
-------------------------------
  U  |  L  |  R  |  U  |  U  |
-------------------------------
  D  |  D  |  U  |  D  |  L  |
-------------------------------
  R  |  D  |     |  U  |  D  |
-------------------------------
  R  |  X  |  L  |  L  |  U  |

Values 169: 
-----------------------------------
-0.00|-0.00|-0.00|-0.00|-0.00|
-----------------------------------
-0.00|-0.00|-0.00|-0.00| 20.00|
-----------------------------------
 5.00| 10.00|-0.00|-1.25|-2.50|
-----------------------------------
 10.00| 20.00| 0.00|-2.50|-0.00|
--------------------------------

 0.16|-0.31|-0.62|-0.31|-0.02|
-----------------------------------
 0.31| 5.00|-0.31|-0.62|-0.02|
-----------------------------------
 5.00| 10.00|-1.25|-1.25|-0.02|
-----------------------------------
 10.00| 20.00| 0.00| 5.00|-0.00|
-----------------------------------
 10.00| 10.00| 20.00| 10.00|-0.00|

Policy 180: 
-------------------------------
  D  |  D  |  D  |  R  |  L  |
-------------------------------
  R  |  D  |  L  |  R  |  U  |
-------------------------------
  D  |  X  |  L  |  D  |  U  |
-------------------------------
  U  |  X  |     |  D  |  L  |
-------------------------------
  U  |  U  |  X  |  L  |  U  |

Values 181: 
-----------------------------------
-0.00|-0.00|-0.00| 20.00| 10.00|
-----------------------------------
-0.00|-0.00|-0.00| 10.00| 20.00|
-----------------------------------
-0.00|-0.00|-0.00|-0.08| 10.00|
-----------------------------------
-0.00|-0.00| 0.00|-0.16|-0.16|
-----------------------------------
-0.00|-0.00|-0.31|-0.31|-0.00|

Policy 181

Values 194: 
-----------------------------------
 1.25| 0.62|-20.00| 20.00| 2.50|
-----------------------------------
 2.50| 1.25| 2.50| 10.00| 20.00|
-----------------------------------
 1.25| 10.00| 5.00| 2.50| 10.00|
-----------------------------------
 2.50| 20.00| 0.00| 0.02| 1.25|
-----------------------------------
 5.00| 10.00| 5.00| 10.00| 5.00|

Policy 194: 
-------------------------------
  D  |  D  |  R  |  X  |  D  |
-------------------------------
  D  |  D  |  R  |  U  |  X  |
-------------------------------
  R  |  D  |  L  |  U  |  U  |
-------------------------------
  R  |  D  |     |  D  |  U  |
-------------------------------
  R  |  U  |  L  |  L  |  U  |

Values 195: 
-----------------------------------
 0.00| 0.01|-0.00|-0.00|-0.00|
-----------------------------------
 0.01| 5.00|-20.00|-0.00|-0.00|
-----------------------------------
 5.00| 10.00| 0.02|-0.00|-0.00|
-----------------------------------
 10.00| 20.00| 0.00| 0.02|-10.00|
---------------------------